In [42]:
import openai
import os

openai.api_key  = os.environ.get('OPENAI_API_KEY')

In [43]:
import json

# It is supposed to be a real API call here
# But we will just return a dummy response
def get_current_weather(location, unit="fahrenheit"):
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }

    # json.dumps() function will convert a subset of Python objects into a json string.
    return json.dumps(weather_info)

In [44]:
functions = [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": [
              "celsius",
              "fahrenheit"
            ]
          }
        },
        "required": [
          "location"
        ]
      }
    }
  ]

In [45]:
messages = [
    {
        "role":  "user",
        "content": "What's the weather like in Boston?"
    }
]

In [46]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)
response

<OpenAIObject chat.completion id=chatcmpl-8FxzkBvcPKnEEulaSMAu95zqEdwUa at 0x11454cb90> JSON: {
  "id": "chatcmpl-8FxzkBvcPKnEEulaSMAu95zqEdwUa",
  "object": "chat.completion",
  "created": 1698816232,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": null,
        "function_call": {
          "name": "get_current_weather",
          "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
        }
      },
      "finish_reason": "function_call"
    }
  ],
  "usage": {
    "prompt_tokens": 82,
    "completion_tokens": 18,
    "total_tokens": 100
  }
}

Our task is to execute get_current_weather with the provided arguments. OpenAI does not execute the function. Instead, it’s the job of our chatbot to run it and parse the returned data.

In [47]:
temp = response.choices[0]["message"].function_call.arguments

arguments = json.loads(temp)

arguments['location']

'Boston, MA'

In [48]:
messages.append(response.choices[0]["message"])
messages.append({
    "role": "function",
    "name": "get_current_weather",
    "content": get_current_weather(arguments['location'])
})

messages

[{'role': 'user', 'content': "What's the weather like in Boston?"},
 <OpenAIObject at 0x1165dcfa0> JSON: {
   "role": "assistant",
   "content": null,
   "function_call": {
     "name": "get_current_weather",
     "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
   }
 },
 {'role': 'function',
  'name': 'get_current_weather',
  'content': '{"location": "Boston, MA", "temperature": "72", "unit": "fahrenheit", "forecast": ["sunny", "windy"]}'}]

In [49]:
response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)
response.choices[0]["message"]["content"]

'The current weather in Boston, MA is 72 degrees Fahrenheit and sunny with windy conditions.'